> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [ ]:
### Escreva sua resposta aqui
import pandas as pd

# URL do arquivo CSV no GitHub
url = "https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/salaries.csv"

# Carregar os dados
df = pd.read_csv(url)

# Mostrar as primeiras 5 linhas para dar uma olhada nos dados
print(df.head())

# Mostrar informações gerais do DataFrame
print(df.info())

#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [ ]:
### Escreva sua resposta aqui
import pandas as pd

url = "https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/salaries.csv"
df = pd.read_csv(url)

print("SEX únicos:", df['SEX'].unique())
print("DESIGNATION únicos:", df['DESIGNATION'].unique())
print("UNIT únicos:", df['UNIT'].unique())

In [ ]:

from sqlalchemy import create_engine, Column, Integer, String, Float, Date, Enum
from sqlalchemy.orm import declarative_base
import enum

Base = declarative_base()

class SexEnum(enum.Enum):
    M = "M"
    F = "F"

class DesignationEnum(enum.Enum):
    Analyst = "Analyst"
    Senior_Analyst = "Senior Analyst"
    Manager = "Manager"
    # Adicione outros cargos conforme necessário

class UnitEnum(enum.Enum):
    IT = "IT"
    Finance = "Finance"
    Marketing = "Marketing"
    # Adicione outras unidades conforme necessário

class SalaryRecord(Base):
    __tablename__ = 'salary_records'

    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String)
    last_name = Column(String)
    sex = Column(Enum(SexEnum))
    doj = Column(Date)  # Date of Joining
    current_date = Column(Date)
    designation = Column(Enum(DesignationEnum))
    age = Column(Integer)
    salary = Column(Float)
    unit = Column(Enum(UnitEnum))
    leaves_used = Column(Integer)
    leaves_remaining = Column(Integer)
    ratings = Column(Float)
    past_exp = Column(Float)

In [ ]:
from sqlalchemy import create_engine

# Cria um engine SQLite na memória (ou você pode usar um arquivo, ex: sqlite:///meubanco.db)
engine = create_engine('sqlite:///:memory:', echo=True)

# Cria as tabelas no banco conforme o modelo definido
Base.metadata.create_all(engine)

print("Banco e tabela criados com sucesso!")

#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [ ]:
### Escreva sua resposta aqui

from sqlalchemy import create_engine

engine = create_engine('sqlite:///salarios.db')

print("Conexão com o banco 'salarios.db' criada com sucesso!")

#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [ ]:

### Escreva sua resposta aqui

# Supondo que você já tenha o seguinte:

from sqlalchemy import create_engine

# engine criado no Q3
engine = create_engine('sqlite:///salarios.db')

# E as classes de modelo definidas e Base declarada (Q2)

# Agora, cria as tabelas no banco
Base.metadata.create_all(engine)

print("Tabelas criadas no banco 'salarios.db' com sucesso!")

#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [ ]:

# Q5 - Populando o banco 'salarios.db' com pandas.to_sql (if_exists='append')
import pandas as pd
from sqlalchemy import create_engine
import sys

# 1 Crie/aponte o engine para o seu SQLite (arquivo salarios.db será criado se não existir)
engine = create_engine('sqlite:///salarios.db')

# 2 Carregar o CSV (do repositório)
url = "https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/salaries.csv"
df = pd.read_csv(url)

# 3 Mostrar colunas originais para debug (opcional)
print("Colunas originais:", df.columns.tolist())

# 4 Renomear automaticamente as colunas mais prováveis para o padrão snake_case usado no ORM
renames = {}
for col in df.columns:
    c = col.lower()
    if 'first' in c and 'name' in c:
        renames[col] = 'first_name'
    elif 'last' in c and 'name' in c:
        renames[col] = 'last_name'
    elif c.strip() in ('sex','gender'):
        renames[col] = 'sex'
    elif 'doj' in c or ('join' in c and 'date' in c) or ('date' in c and 'joining' in c):
        renames[col] = 'doj'
    elif 'current' in c and 'date' in c:
        renames[col] = 'current_date'
    elif 'design' in c:
        renames[col] = 'designation'
    elif 'age' == c or c.startswith('age'):
        renames[col] = 'age'
    elif 'salary' in c:
        renames[col] = 'salary'
    elif 'unit' in c or 'department' in c or 'business' in c:
        renames[col] = 'unit'
    elif 'leave' in c and 'used' in c:
        renames[col] = 'leaves_used'
    elif 'leave' in c and ('remain' in c or 'remaining' in c):
        renames[col] = 'leaves_remaining'
    elif 'rating' in c:
        renames[col] = 'ratings'
    elif 'past' in c and ('exp' in c or 'experience' in c):
        renames[col] = 'past_exp'

# Aplicar renomeação e mostrar mapeamento
df.rename(columns=renames, inplace=True)
print("Renomeações aplicadas:", renames)
print("Colunas após rename:", df.columns.tolist())

# 5 Converter colunas de data (quando existirem). Usamos errors='coerce' para evitar crashes.
if 'doj' in df.columns:
    df['doj'] = pd.to_datetime(df['doj'], errors='coerce')
if 'current_date' in df.columns:
    df['current_date'] = pd.to_datetime(df['current_date'], errors='coerce')

# 6 Converter colunas numéricas (quando existirem)
num_cols = ['age','salary','leaves_used','leaves_remaining','ratings','past_exp']
for nc in num_cols:
    if nc in df.columns:
        df[nc] = pd.to_numeric(df[nc], errors='coerce')

# 7 Limpar/normalizar strings (sex, designation, unit)
for s in ('sex','designation','unit'):
    if s in df.columns:
        df[s] = df[s].astype(str).str.strip()

# 8 Antes de inserir: contar linhas já existentes na tabela (se existir)
table_name = 'salary_records'
try:
    before = pd.read_sql_query(f"SELECT COUNT(*) AS cnt FROM {table_name}", engine).iloc[0,0]
except Exception:
    before = 0
print(f"Linhas antes da inserção na tabela '{table_name}': {before}")

# 9 Inserir usando to_sql com if_exists='append' (não altera a tabela existente)
#    index=False para não criar coluna index como campo na tabela
df.to_sql(table_name, con=engine, if_exists='append', index=False)

# 10 Confirmar quantas linhas agora existem
after = pd.read_sql_query(f"SELECT COUNT(*) AS cnt FROM {table_name}", engine).iloc[0,0]
print(f"Linhas depois da inserção na tabela '{table_name}': {after}")
print(f"{after - before} linhas inseridas (esperado: {len(df)}).")

# 11 Mostrar as primeiras 5 linhas inseridas (query)
print("\nPrimeiras 5 linhas da tabela (preview):")
preview = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 5", engine)
print(preview)

#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [ ]:
!pip install pandas sqlalchemy

In [ ]:
# Célula 1 — imports e engine
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.orm import declarative_base

# engine apontando para o arquivo SQLite (salarios.db)
engine = create_engine('sqlite:///salarios.db', echo=False)

# teste rápido
print("Engine criado:", engine)

In [ ]:
# Célula 2 — modelo ORM (opcional se você já tem)
from sqlalchemy import Column, Integer, String, Float, Date, Enum
import enum

Base = declarative_base()

class SexEnum(enum.Enum):
    M = "M"
    F = "F"

# simplifiquei os enums — adicione mais se necessário
class DesignationEnum(enum.Enum):
    Analyst = "Analyst"
    Senior_Analyst = "Senior Analyst"
    Manager = "Manager"

class UnitEnum(enum.Enum):
    IT = "IT"
    Finance = "Finance"
    Marketing = "Marketing"

class SalaryRecord(Base):
    __tablename__ = 'salary_records'   # <--- certifique-se que bate com o nome usado no DB
    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String)
    last_name = Column(String)
    sex = Column(String)          # coloquei String para evitar problemas com Enum ao inserir via to_sql
    doj = Column(Date)
    current_date = Column(Date)
    designation = Column(String)
    age = Column(Integer)
    salary = Column(Float)
    unit = Column(String)
    leaves_used = Column(Integer)
    leaves_remaining = Column(Integer)
    ratings = Column(Float)
    past_exp = Column(Float)

# só cria as tabelas se quiser garantir (não sobrescreve se já existirem)
Base.metadata.create_all(engine)
print("Modelo SalaryRecord definido / tabelas criadas.")

In [ ]:
# Célula 3 — checar contagem / preview da tabela
table = 'salary_records'   # ajuste se seu nome for diferente
try:
    df_preview = pd.read_sql_query(f"SELECT COUNT(*) AS cnt FROM {table}", engine)
    print("Contagem atual na tabela:", int(df_preview['cnt'].iloc[0]))
    print("\nPreview (5 linhas):")
    print(pd.read_sql_query(f"SELECT * FROM {table} LIMIT 5", engine))
except Exception as e:
    print("Erro ao acessar a tabela — talvez ela não exista ou o nome esteja diferente.")
    print("Detalhe:", e)

In [ ]:
# Forma C: ORM select() + Session
from sqlalchemy import select, func
from sqlalchemy.orm import Session

with Session(engine) as session:
    stmt = (
        select(
            SalaryRecord.designation,
            func.min(SalaryRecord.salary / 12.0).label("salario_min_mensal"),
            func.max(SalaryRecord.salary / 12.0).label("salario_max_mensal"),
            func.avg(SalaryRecord.salary / 12.0).label("salario_medio_mensal")
        )
        .group_by(SalaryRecord.designation)
        .order_by(SalaryRecord.designation)
    )
    result = session.execute(stmt).all()

# transformar em DataFrame
df_c = pd.DataFrame(result, columns=["designation","salario_min_mensal","salario_max_mensal","salario_medio_mensal"])
print(df_c)